In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchio as tio
from torchsummary import summary
import torchvision
import torchvision.transforms as transforms


from modules import Evaluator2D, Dataset, MRIDataset, MRIDatasets, Trainer2D, UNet2D
from modules.LossFunctions import DC_and_CE_loss, GDiceLossV2
from modules.Tensorboard import TensorboardModules
from modules.Utils import *

RuntimeError: KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
params = {"total_epochs": 100,
          "batch_size": 1,
          "SGD": {"lr": 1e-01, "momentum": 0.9, "nesterov": True},
          #"AdamW": {"lr": 1e-01, "betas":(0.9, 0.999), "eps": 1e-8},       
          "ES":{"patience": 20, "min_delta": 1e-03},
          "CLR":{"base": 1e-05, "max": 1e-02, "up": 3, "down": 5, "mode": "triangular2"},
          #"SLR":{'step_size': 13, "gamma": 1e-1}          
         }

output_path = "output/UNet2D/Iteration1/run1"
weight_path = os.path.join(output_path, "weights/")

In [ ]:
# There are multiple data folders belong to same dataset. Each of them processed in different ways.
# Therefore, path of the data and its name explicitly are defined.
dataset_train = MRIDatasets.FeTA_BalancedDistribution
dataset_path_train = os.path.join('data', 'feta_2.1')
dataset_val = MRIDatasets.FeTA_BalancedDistribution
dataset_path_val = os.path.join('data', 'feta_2.1')

#cv_ = "cv3" # 5-fold cross-validation. Folds [cv1-cv5]

# Transformations.
transform_train = transforms.Compose([tio.RandomMotion(), tio.RandomNoise(), 
                                      tio.RandomAffine(degrees=[15], isotropic=True)])

transform_eval = None # transforms.Compose([Mask()])


# Split dataset.
train = MRIDataset(dataset_train, "train", dataset_path_train, transform=transform_train)
val = MRIDataset(dataset_val, "val", dataset_path_val, transform=transform_eval)
test = MRIDataset(dataset_val, "test", dataset_path_val, transform=transform_eval)

torch.manual_seed(0)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=params["batch_size"], num_workers=0, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=params["batch_size"])
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=params["batch_size"])

# Add dataset configuration to parameters to save them as meta data.
params["dataset_train"] = dataset_train.name
params["dataset_path_train"] = dataset_path_train
params["dataset_val"] = dataset_val.name
params["dataset_path_val"] = dataset_path_val
#params["cross_validation"] = "None" if not cv_ else cv_
params["transform_train"] = "None" if not transform_train else str(transform_train.transforms)
params["transform_eval"] = "None" if not transform_eval else str(transform_eval.transforms)

In [ ]:
model = UNet2D().to(device)
criterion = DC_and_CE_loss({'batch_dice': True, 'smooth': 1e-5, 'do_bg': False, 'square': False}, {})
pretrained = False

# Initalize weights or load already trained model.
if not pretrained:
    model.apply(init_weights_kaiming)
    params["initial_weights"] = init_weights_kaiming.__name__
else:
    model_path = "output/UNet3D/Iteration9/run8/weights/32_model.pth"
    model.load_state_dict(torch.load(model_path))
    params["initial_weights"] = model_path

In [ ]:
#print(summary(model, input_size=(1, 128, 128, 128)))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=params["SGD"]["lr"], 
                            momentum=params["SGD"]["momentum"], nesterov=params["SGD"]["nesterov"])

# optimizer = torch.optim.AdamW(model.parameters(), lr=params["AdamW"]["lr"], 
#                              betas=params["AdamW"]["betas"], eps=params["AdamW"]["eps"])


scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=params["CLR"]["base"], 
                                              max_lr=params["CLR"]["max"],
                                              step_size_up=params["CLR"]["up"], 
                                              step_size_down=params["CLR"]["down"],
                                              mode=params["CLR"]["mode"],
                                              cycle_momentum=False)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=params["SLR"]["step_size"], 
#                                            gamma=params["SLR"]["gamma"])

early_stopping = EarlyStopping(patience=params["ES"]["patience"], min_delta=params["ES"]["min_delta"])


# Initalize trainer for training.
trainer = Trainer2D(criterion, model, optimizer, params["total_epochs"], train_loader, scheduler)

# Initalize evaluator for validation.
evaluator = Evaluator2D(criterion, model, val_loader)

In [ ]:
# Create output and path if it is not exist.
if not os.path.isdir(weight_path):
    os.makedirs(weight_path)

# Create Tensorboard object to save experiment outputs.    
tb = TensorboardModules(output_path)

# Save hyperparameters as note.
(pd.DataFrame.from_dict(data=params, orient='index')
 .to_csv(os.path.join(output_path,"details.txt"), header=False, sep="="))

# Add some images and corresponding masks into Tensorboard.
mri_image, mri_mask = val[0]['mri']['data'].squeeze(0), val[0]['mask']['data'].squeeze(0)
slices = (50, 100, 10)
tb.add_image_mask(mri_image, mri_mask, slices)

# Add model graph to Tensorboard.
tb.add_graph(model, (256, 256), device)
# print(summary(model, input_size=(1, 32, 128, 128)))

In [ ]:
prev_weights = ""
prev_val_loss = 100

for epoch in range(0, params["total_epochs"]):
    # One forward pass for all training data.
    avg_train_loss = trainer.fit()
    
    # Evaluate current model on validation data.
    avg_val_loss, dice_scores = evaluator.evaluate()
    avg_scores = sum(dice_scores) / len(dice_scores)
    
    print("-------------------------------------------------------------")
    
    # Add results to tensorboard.
    tb.add_scalars(step=epoch+1, lr=scheduler.get_last_lr()[0], ds=avg_scores, 
                   train_loss=avg_train_loss, val_loss=avg_val_loss)
    
    model_name = "_".join([str(epoch), "model.pth"])
    model_path = os.path.join(weight_path, model_name)
    
    if avg_val_loss < prev_val_loss:
        # Save trained weights.
        if os.path.isfile(prev_weights):
            os.remove(prev_weights)        
        torch.save(model.state_dict(), model_path)
        
    prev_weights = model_path        
    prev_val_loss = avg_val_loss
    
    # If model is not learning stop the training.
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        break

print('Finished Training')